In [1]:
import numpy as np 
import pandas as pd 

from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)

import warnings
warnings.simplefilter(action='ignore')
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [2]:
train_df = pd.read_csv(r"C:\Users\20943\Desktop\我的坚果云\Sys_learning\3_\titanic\train.csv")
test_df = pd.read_csv(r"C:\Users\20943\Desktop\我的坚果云\Sys_learning\3_\titanic\test.csv")

In [3]:
help(BaggingClassifier)

Help on class BaggingClassifier in module sklearn.ensemble._bagging:

class BaggingClassifier(sklearn.base.ClassifierMixin, BaseBagging)
 |  BaggingClassifier(base_estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)
 |  
 |  A Bagging classifier.
 |  
 |  A Bagging classifier is an ensemble meta-estimator that fits base
 |  classifiers each on random subsets of the original dataset and then
 |  aggregate their individual predictions (either by voting or by averaging)
 |  to form a final prediction. Such a meta-estimator can typically be used as
 |  a way to reduce the variance of a black-box estimator (e.g., a decision
 |  tree), by introducing randomization into its construction procedure and
 |  then making an ensemble out of it.
 |  
 |  This algorithm encompasses several works from the literature. When random
 |  subsets of the dataset are drawn

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_data = train_df.copy()
train_data["Age"].fillna(train_df["Age"].median(skipna=True), inplace=True)
train_data["Embarked"].fillna(train_df['Embarked'].value_counts().idxmax(), inplace=True)
train_data.drop('Cabin', axis=1, inplace=True)
train_data['TravelAlone']=np.where((train_data["SibSp"]+train_data["Parch"])>0, 0, 1)
train_data.drop('SibSp', axis=1, inplace=True)
train_data.drop('Parch', axis=1, inplace=True)
training=pd.get_dummies(train_data, columns=["Pclass","Embarked","Sex"])#进行独热编码
training.drop('Sex_female', axis=1, inplace=True)
training.drop('PassengerId', axis=1, inplace=True)
training.drop('Name', axis=1, inplace=True)
training.drop('Ticket', axis=1, inplace=True)

test_data = test_df.copy()
test_data["Age"].fillna(train_df["Age"].median(skipna=True), inplace=True)
test_data["Fare"].fillna(train_df["Fare"].median(skipna=True), inplace=True)
test_data.drop('Cabin', axis=1, inplace=True)
test_data['TravelAlone']=np.where((test_data["SibSp"]+test_data["Parch"])>0, 0, 1)
test_data.drop('SibSp', axis=1, inplace=True)
test_data.drop('Parch', axis=1, inplace=True)
testing = pd.get_dummies(test_data, columns=["Pclass","Embarked","Sex"])
testing.drop('Sex_female', axis=1, inplace=True)
testing.drop('PassengerId', axis=1, inplace=True)
testing.drop('Name', axis=1, inplace=True)
testing.drop('Ticket', axis=1, inplace=True)
final_test = testing
final_train = training

final_train['IsMinor']=np.where(final_train['Age']<=16, 1, 0)
final_test['IsMinor']=np.where(final_test['Age']<=16, 1, 0)

final_train.head()

,Survived,Age,Fare,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male,IsMinor
0,0,22.0,7.2500,0,0,0,1,0,0,1,1,0
1,1,38.0,71.2833,0,1,0,0,1,0,0,0,0
2,1,26.0,7.9250,1,0,0,1,0,0,1,0,0
3,1,35.0,53.1000,0,1,0,0,0,0,1,0,0
4,0,35.0,8.0500,1,0,0,1,0,0,1,1,0


In [4]:
cols = ["Age","Fare","TravelAlone","Pclass_1","Pclass_2","Embarked_C","Embarked_S","Sex_male","IsMinor"] 
X = final_train[cols]
y = final_train['Survived']

In [7]:
superpa = []
for i in range(200):
    rfc = RandomForestClassifier(n_estimators=i+1,n_jobs=-1)
    rfc_s = cross_val_score(rfc,X,y,cv=10).mean()
    superpa.append(rfc_s)
print(max(superpa),superpa.index(max(superpa)))
plt.figure(figsize=[20,5])
plt.plot(range(1,201),superpa)
plt.show()

KeyboardInterrupt: 

In [ ]:
superpa = []
for i in range(200):
    rfc = RandomForestClassifier(n_estimators=13,max_depth = i+1, n_jobs=-1)
    rfc_s = cross_val_score(rfc,X,y,cv=10).mean()
    superpa.append(rfc_s)
print(max(superpa),superpa.index(max(superpa)))
plt.figure(figsize=[20,5])
plt.plot(range(1,201),superpa)
plt.show()

In [ ]:
superpa = []
for i in range(1000):
    rfc = RandomForestClassifier(n_estimators=13,max_depth = 86, max_leaf_nodes = i+1)
    rfc_s = cross_val_score(rfc,X,y,cv=10).mean()
    superpa.append(rfc_s)
print(max(superpa),superpa.index(max(superpa)))
plt.figure(figsize=[20,5])
plt.plot(range(1,1001),superpa)
plt.show()

In [ ]:
'''
param_grid={'min_samples_leaf':np.arange(1, 1+30, 1)
           ,'min_samples_split':np.arange(2, 2+30, 1)
           ,'criterion':['gini', 'entropy']
           ,'max_features':np.arange(1,100,1)
           ,'max_leaf_nodes': np.arange(1,200,1)}
rfc = RandomForestClassifier(n_estimators=13,max_depth = 86,random_state = 0)
GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(X,y)
GS.best_params_
GS.best_score_
'''

In [41]:
rf = RandomForestClassifier(random_state = 0
                            ,n_estimators = 13
                            ,max_depth = 86
                            ,min_samples_leaf = 3
                            ,min_samples_split = 10
                            ,max_features = 7
                            ,max_leaf_nodes = 55
                            ,criterion = 'gini'
                            )
lr = LogisticRegression(C=4.1,random_state = 0)
eclf1 = VotingClassifier(estimators=[('lr', lr), ('rf', rf)],voting = 'soft')
eclf = eclf1.fit(X,y)
rf = rf.fit(X,y)
lr = lr.fit(X,y)

In [11]:
def get_models():
    """Generate a library of base learners."""
   #nb = GaussianNB()
   #svc = SVC(C=100, probability=True)
   #knn = KNeighborsClassifier(n_neighbors=3)
    lr = LogisticRegression(C=4.1,random_state = 0)
   #nn = MLPClassifier((80, 10), early_stopping=False, random_state=SEED)
   #gb = GradientBoostingClassifier(n_estimators=100, random_state=SEED)
    rf = RandomForestClassifier(random_state = 0
                            ,n_estimators = 13
                            ,max_depth = 86
                            ,min_samples_leaf = 3
                            ,min_samples_split = 10
                            ,max_features = 7
                            ,max_leaf_nodes = 55
                            ,criterion = 'gini'
                            )

    models = {#'svm': svc,
#'knn': knn,
#'naive bayes': nb,
#'mlp-nn': nn,
'random forest': rf,
#'gbm': gb,
'logistic': lr,
             }
    return models


In [12]:
def train_predict(model_list):
    """Fit models in list on training set and return preds"""
    P = np.zeros((ytest.shape[0], len(model_list)))
    P = pd.DataFrame(P)

    print("Fitting models.")
    cols = list()
    for i, (name, m) in enumerate(models.items()):
        print("%s..." % name, end=" ", flush=False)
        m.fit(xtrain, ytrain)
        P.iloc[:, i] = m.predict_proba(xtest)[:, 1]
        cols.append(name)
    print("done")
    P.columns = cols
    print("Done.\n")
    return P


def score_models(P, y):
    """Score model in prediction DF"""
    print("Scoring models.")
    for m in P.columns:
        score = roc_auc_score(y, P.loc[:, m])
    print("%-26s: %.3f" % (m, score))
    print("Done.\n")

In [13]:
base_learners = get_models()
meta_learner = GradientBoostingClassifier(
   n_estimators=1000,
   loss="exponential",
   #max_features=4,
   max_depth=3,
   subsample=0.5,
   learning_rate=0.005, 
   random_state=0
)

In [25]:
def train_base_learners(base_learners, inp, out, verbose=True):
    """Train all base learners in the library."""
    if verbose: print("Fitting models.")
    for i, (name, m) in enumerate(base_learners.items()):
        if verbose: 
            print("%s..." % name, end=" ", flush=False)
            m.fit(inp, out)
        if verbose: 
            print("done")

def predict_base_learners(pred_base_learners, inp, verbose=True):
    """Generate a prediction matrix."""
    P = np.zeros((inp.shape[0], len(pred_base_learners)))
    if verbose: 
        print("Generating base learner predictions.")
    for i, (name, m) in enumerate(pred_base_learners.items()):
        if verbose: 
            print("%s..." % name, end=" ", flush=False)
            p = m.predict_proba(inp)# With two classes, need only predictions for one class
            P[:, i] = p[:, 1]
        if verbose:
            print("done")
    return P

def ensemble_predict(base_learners, meta_learner, inp, verbose=True):
    """Generate predictions from the ensemble."""
    P_pred = predict_base_learners(base_learners, inp, verbose=verbose)
    return P_pred, meta_learner.predict_proba(P_pred)[:, 1]

In [22]:
xtrain_base, xpred_base, ytrain_base, ypred_base = train_test_split(
   X, y, test_size=0.5, random_state=0)

In [23]:
train_base_learners(base_learners, xtrain_base, ytrain_base)

Fitting models.
random forest... done
logistic... done


In [26]:
P_base = predict_base_learners(base_learners, xpred_base)

Generating base learner predictions.
random forest... done
logistic... done


In [27]:
meta_learner.fit(P_base, ypred_base)
P_pred, p = ensemble_predict(base_learners, meta_learner, final_test[cols])
#print("\nEnsemble ROC-AUC score: %.3f" % roc_auc_score(rf.predict(final_test[cols]), p))

Generating base learner predictions.
random forest... done
logistic... done


In [28]:
P_pred

array([[0.15703186, 0.142315  ],
       [0.19846329, 0.44941786],
       [0.06783217, 0.18587876],
       [0.34830725, 0.08823647],
       [0.82921523, 0.59761491],
       [0.26007326, 0.1575024 ],
       [0.67252471, 0.68775687],
       [0.08550296, 0.232666  ],
       [0.83985961, 0.73384181],
       [0.03525641, 0.1140828 ],
       [0.01398601, 0.08629195],
       [0.23925647, 0.32107611],
       [0.93333333, 0.92089399],
       [0.02457265, 0.11154223],
       [1.        , 0.86603184],
       [0.98076923, 0.87974715],
       [0.13760534, 0.30297698],
       [0.1260355 , 0.17709209],
       [0.3890477 , 0.56783284],
       [0.42204377, 0.59208951],
       [0.38441558, 0.46342964],
       [0.34567484, 0.19257738],
       [0.83928571, 0.89686012],
       [0.34566056, 0.65994896],
       [0.84387386, 0.93532257],
       [0.03525641, 0.06025523],
       [1.        , 0.95766566],
       [0.1260355 , 0.17195734],
       [0.71144454, 0.34837461],
       [0.45407413, 0.17360289],
       [0.

In [33]:
p_predproba = 0.5*(rf.predict_proba(final_test[cols]) + rf.predict_proba(final_test[cols]))

In [39]:
p_pred = []
for i in p_predproba:
    if i[0]>=i[1]:
        p_pred.append(0)
    else:
        p_pred.append(1)
        
p_pred = np.array(p_pred)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [42]:
final_test['Survived'] = eclf.predict(final_test[cols])
final_test['PassengerId'] = test_df['PassengerId']

submission = final_test[['PassengerId','Survived']]
submission.to_csv("submission1.csv", index=False)
submission.tail()

,PassengerId,Survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0
